In [6]:
!pip install openpyxl

In [7]:
import os
import io
import pandas as pd
from rdflib import ConjunctiveGraph
from rdflib.plugins.sparql.results.csvresults import CSVResultSerializer

pd.set_option('display.max_colwidth', 200)

if not os.path.exists('outputs'):
    os.makedirs('outputs')
    
g = ConjunctiveGraph()
g.parse('data.trig', format='trig')

def query(q):
    serializer = CSVResultSerializer(g.query(q))
    output = io.BytesIO()
    serializer.serialize(output)
    return pd.read_csv(io.StringIO(output.getvalue().decode('utf-8')), encoding='utf-8')
    
def write(df, fn):
    df.to_excel('outputs/{}'.format(fn), encoding='utf-8')

In [8]:
q = query("""
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX envri: <http://envri.eu/ns/>
PREFIX rm: <http://www.oil-e.net/ontology/envri-rm.owl#>

SELECT ?date ?ri_acronym ?ri_website ?ri_domain ?rep_label WHERE {
    ?g dcterms:date ?date .
    GRAPH ?g { 
        ?ri a rm:ResearchInfrastructure .
        ?ri envri:acronym ?ri_acronym . 
        ?ri envri:website ?ri_website .
        ?ri envri:hasDomain ?ri_domain .
        ?ri envri:hasRepository ?rep .
        ?rep a envri:Repository .
        ?rep rdfs:label ?rep_label .
    }
    FILTER (?date > "2019-01-01"^^xsd:date)
    # FILTER (?ri_acronym = "ICOS")
}
""")

display(q)
write(q, 'output-1.xlsx')

,date,ri_acronym,ri_website,ri_domain,rep_label
0,2019-03-21,EPOS,http://envri.eu/ns/NULL,n482,RESIF (France)
1,2019-04-26,ACTRIS,http://actris.nilu.no,n199,EARLINET Database
2,2019-04-26,ACTRIS,http://actris.nilu.no,n199,ACTRIS - In-Situ unit
3,2019-04-26,ACTRIS,http://actris.nilu.no,n199,ACTRIS-ACCESS
4,2019-04-01,ICOS,http://www.icos-cp.eu,n5,Carbon Portal
5,2019-04-03,EPOS,http://envri.eu/ns/NULL,n503,European Federated Data Archive
6,2019-04-30,LW (Marine),http://www.euro-argo.eu,n123,Marine Data Archive
7,2019-04-30,LW (Marine),http://www.euro-argo.eu,n123,EUROBIS
8,2019-04-18,Euro-Argo,http://www.euro-argo.eu,n289,Euro-Argo Data
9,2019-04-16,EPOS,http://envri.eu/ns/NULL,n458,Terradue


In [9]:
q = query("""
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX envri: <http://envri.eu/ns/>
PREFIX rm: <http://www.oil-e.net/ontology/envri-rm.owl#>

SELECT ?rep_label ?vocab_name ?vocab_iri WHERE {
    [] a rm:ResearchInfrastructure ;
       envri:hasRepository [
         a envri:Repository ;
         rdfs:label ?rep_label ;
         envri:hasVocabularies [
           envri:hasName ?vocab_name ;
           envri:hasVocabularyIri ?vocab_iri 
         ]
       ]
}
""")

display(q)
write(q, 'output-2.xlsx')

,rep_label,vocab_name,vocab_iri
0,RESIF (France),http://envri.eu/ns/NULL,http://envri.eu/ns/NULL
1,Marine Data Archive,Marine species,http://www.marinespecies.org/
2,Marine Data Archive,WoRMS,http://www.marinespecies.org/
3,Marine Data Archive,Marine Regions,http://www.marineregions.org/
4,EARLINET Database,CF Standard Names,http://cfconventions.org/
5,ACTRIS - In-Situ unit,CF standard names,http://cfconventions.org/latest.html
6,EUROBIS,WoRMS,http://www.marinespecies.org/
7,EUROBIS,Marine species,http://www.marinespecies.org/
8,EUROBIS,Marine Regions,http://www.marineregions.org/
9,SeaDataNet Central Data Products,NERC vocabulary service,http://vocab.nerc.ac.uk/collection/V22/current/


In [10]:
q = query("""
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX envri: <http://envri.eu/ns/>
PREFIX rm: <http://www.oil-e.net/ontology/envri-rm.owl#>
PREFIX fairterms: <https://w3id.org/fair/principles/terms/>

SELECT ?p WHERE {
    ?p envri:relatesTo fairterms:A1.2 .
}
""")

display(q)
write(q, 'output-3.xlsx')

,p
0,http://envri.eu/ns/contentAccessAuthorizationRequired
1,http://envri.eu/ns/hasAuthenticationMethod
2,http://envri.eu/ns/usesAuthorisationTechnique
3,http://envri.eu/ns/maintainsOwnUserDatabase


In [11]:
q = query("""
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX envri: <http://envri.eu/ns/>
PREFIX rm: <http://www.oil-e.net/ontology/envri-rm.owl#>
PREFIX fairterms: <https://w3id.org/fair/principles/terms/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

SELECT ?l ?p ?o ?li WHERE {
    ?p envri:relatesTo fairterms:I1 .
    ?s ?p ?o .
    ?s skos:altLabel ?l .
    OPTIONAL { ?o a rdf:Bag . ?o rdf:li ?li }
    FILTER (?o != envri:NULL)
}
""")

display(q)
write(q, 'output-4.xlsx')

,l,p,o,li
0,ICOS Carbon Portal vocabularies,http://envri.eu/ns/hasTopic,http://envri.eu/ns/General,NaN
1,SDN SeaDataNet Common DAta Index (CDI) vocabularies,http://envri.eu/ns/hasTopic,http://envri.eu/ns/INSPIRE,NaN
2,LW (Marine) Marine Data Archive vocabularies,http://envri.eu/ns/hasTopic,http://envri.eu/ns/DomainSpecific,NaN
3,SDN SeaDataNet Common DAta Index (CDI) vocabularies,http://envri.eu/ns/hasTopic,http://envri.eu/ns/DomainSpecific,NaN
4,Euro-Argo Euro-Argo Data vocabularies,http://envri.eu/ns/hasTopic,http://envri.eu/nx/ProjectSpecific,NaN
5,SDN SeaDataNet Common DAta Index (CDI) vocabularies,http://envri.eu/ns/hasTopic,http://envri.eu/nx/ProjectSpecific,NaN
6,SDN SeaDataNet Common DAta Index (CDI) vocabularies,http://envri.eu/ns/hasTopic,http://envri.eu/nx/ProjectSpecific,NaN
7,SDN SeaDataNet Central Data Products vocabularies,http://envri.eu/ns/hasTopic,http://envri.eu/ns/INSPIRE,NaN
8,SDN SeaDataNet Central Data Products vocabularies,http://envri.eu/ns/hasTopic,http://envri.eu/nx/ProjectSpecific,NaN
9,SDN SeaDataNet Central Data Products vocabularies,http://envri.eu/ns/hasTopic,http://envri.eu/ns/INSPIRE,NaN


In [12]:
q = query("""
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX envri: <http://envri.eu/ns/>
PREFIX rm: <http://www.oil-e.net/ontology/envri-rm.owl#>

SELECT ?rep_label ?schema_name WHERE {
    [] a rm:ResearchInfrastructure ;
       envri:hasRepository [
         a envri:Repository ;
         rdfs:label ?rep_label ;
         envri:hasMetadata [
           envri:hasSchema [
             envri:hasSchemaName ?schema_name
           ]
         ]
       ]
}
""")

display(q)
write(q, 'output-5.xlsx')

,rep_label,schema_name
0,RESIF (France),http://envri.eu/ns/StationXML
1,Marine Data Archive,http://envri.eu/ns/EML
2,Marine Data Archive,http://envri.eu/ns/ISO_19115
3,EARLINET Database,http://envri.eu/ns/ISO_19115
4,ACTRIS - In-Situ unit,http://envri.eu/ns/ISO_19115
5,EUROBIS,http://envri.eu/ns/EML
6,EUROBIS,http://envri.eu/ns/ISO_19115
7,SeaDataNet Central Data Products,http://envri.eu/ns/ISO_19115_19139
8,SeaDataNet Central Data Products,http://envri.eu/ns/SeaDataNetCommunityProfile
9,SeaDataNet Common DAta Index (CDI),http://envri.eu/ns/ISO_19115_19139


In [13]:
q = query("""
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX envri: <http://envri.eu/ns/>
PREFIX rm: <http://www.oil-e.net/ontology/envri-rm.owl#>
PREFIX fairterms: <https://w3id.org/fair/principles/terms/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

SELECT ?l ?p ?o ?li WHERE {
    ?p envri:relatesTo fairterms:I1 .
    ?s ?p ?o .
    ?s skos:altLabel ?l .
    OPTIONAL { ?o a rdf:Bag . ?o rdf:li ?li }
    FILTER (?o != envri:NULL)
}
""")

display(q)
write(q, 'output-I1.xlsx')

,l,p,o,li
0,ICOS Carbon Portal vocabularies,http://envri.eu/ns/hasTopic,http://envri.eu/ns/General,NaN
1,SDN SeaDataNet Common DAta Index (CDI) vocabularies,http://envri.eu/ns/hasTopic,http://envri.eu/ns/INSPIRE,NaN
2,LW (Marine) Marine Data Archive vocabularies,http://envri.eu/ns/hasTopic,http://envri.eu/ns/DomainSpecific,NaN
3,SDN SeaDataNet Common DAta Index (CDI) vocabularies,http://envri.eu/ns/hasTopic,http://envri.eu/ns/DomainSpecific,NaN
4,Euro-Argo Euro-Argo Data vocabularies,http://envri.eu/ns/hasTopic,http://envri.eu/nx/ProjectSpecific,NaN
5,SDN SeaDataNet Common DAta Index (CDI) vocabularies,http://envri.eu/ns/hasTopic,http://envri.eu/nx/ProjectSpecific,NaN
6,SDN SeaDataNet Common DAta Index (CDI) vocabularies,http://envri.eu/ns/hasTopic,http://envri.eu/nx/ProjectSpecific,NaN
7,SDN SeaDataNet Central Data Products vocabularies,http://envri.eu/ns/hasTopic,http://envri.eu/ns/INSPIRE,NaN
8,SDN SeaDataNet Central Data Products vocabularies,http://envri.eu/ns/hasTopic,http://envri.eu/nx/ProjectSpecific,NaN
9,SDN SeaDataNet Central Data Products vocabularies,http://envri.eu/ns/hasTopic,http://envri.eu/ns/INSPIRE,NaN


In [14]:
q = query("""
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX envri: <http://envri.eu/ns/>
PREFIX rm: <http://www.oil-e.net/ontology/envri-rm.owl#>
PREFIX fairterms: <https://w3id.org/fair/principles/terms/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

SELECT ?l ?p ?o ?li WHERE {
    ?p envri:relatesTo fairterms:A1 .
    ?s ?p ?o .
    ?s skos:altLabel ?l .
    OPTIONAL { ?o a rdf:Bag . ?o rdf:li ?li }
    FILTER (?o != envri:NULL)
}
""")

display(q)
write(q, 'output-A1.xlsx')

,l,p,o,li
0,SDN SeaDataNet Central Data Products data processing,http://envri.eu/ns/otherAnalysisServicesOffered,http://envri.eu/ns/Planned,NaN
1,EPOS European Federated Data Archive data processing,http://envri.eu/ns/otherAnalysisServicesOffered,http://envri.eu/ns/None,NaN
2,ACTRIS EARLINET Database data processing,http://envri.eu/ns/otherAnalysisServicesOffered,n272,http://envri.eu/ns/DOIGeneration
3,ACTRIS EARLINET Database data processing,http://envri.eu/ns/otherAnalysisServicesOffered,n272,http://envri.eu/ns/QA_QC
4,SDN SeaDataNet Central Data Products data processing,http://envri.eu/ns/otherAnalysisServicesOffered,http://envri.eu/ns/Planned,NaN
5,ICOS Carbon Portal data processing,http://envri.eu/ns/otherAnalysisServicesOffered,n26,http://envri.eu/ns/JupiterNotebooks
6,SDN SeaDataNet Common DAta Index (CDI) data processing,http://envri.eu/ns/otherAnalysisServicesOffered,http://envri.eu/ns/Planned,NaN
7,LW (Marine) EUROBIS data processing,http://envri.eu/ns/otherAnalysisServicesOffered,n175,http://envri.eu/ns/RPackages
8,LW (Marine) Marine Data Archive data processing,http://envri.eu/ns/otherAnalysisServicesOffered,http://envri.eu/ns/None,NaN
9,EPOS Terradue data processing,http://envri.eu/ns/otherAnalysisServicesOffered,http://envri.eu/ns/None,NaN


In [15]:
q = query("""
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX envri: <http://envri.eu/ns/>
PREFIX rm: <http://www.oil-e.net/ontology/envri-rm.owl#>
PREFIX fairterms: <https://w3id.org/fair/principles/terms/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

SELECT ?l ?p ?o ?li WHERE {
    ?p envri:relatesTo fairterms:A2 .
    ?s ?p ?o .
    ?s skos:altLabel ?l .
    OPTIONAL { ?o a rdf:Bag . ?o rdf:li ?li }
    FILTER (?o != envri:NULL)
}
""")

display(q)
write(q, 'output-A2.xlsx')

,l,p,o,li


In [16]:
q = query("""
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX envri: <http://envri.eu/ns/>
PREFIX rm: <http://www.oil-e.net/ontology/envri-rm.owl#>
PREFIX fairterms: <https://w3id.org/fair/principles/terms/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

SELECT ?l ?p ?o ?li WHERE {
    ?p envri:relatesTo fairterms:A1.1 .
    ?s ?p ?o .
    ?s skos:altLabel ?l .
    OPTIONAL { ?o a rdf:Bag . ?o rdf:li ?li }
    FILTER (?o != envri:NULL)
}
""")

display(q)
write(q, 'output-A1_1.xlsx')

,l,p,o,li
0,SDN SeaDataNet Common DAta Index (CDI) access mechanism,http://envri.eu/ns/supportsAccessTechnology,http://envri.eu/ns/CAS,NaN
1,LW (Marine) EUROBIS access mechanism,http://envri.eu/ns/supportsAccessTechnology,http://envri.eu/ns/HTTPPOST,NaN
2,EMSO EMSO data access mechanism,http://envri.eu/ns/supportsAccessTechnology,http://envri.eu/ns/HTTP,NaN
3,LW (Marine) Marine Data Archive access mechanism,http://envri.eu/ns/supportsAccessTechnology,http://envri.eu/ns/HTTPPOST,NaN
4,SDN SeaDataNet Central Data Products access mechanism,http://envri.eu/ns/supportsAccessTechnology,http://envri.eu/ns/CAS,NaN
5,Euro-Argo Euro-Argo Data access mechanism,http://envri.eu/ns/supportsAccessTechnology,http://envri.eu/ns/FTP,NaN
6,ACTRIS ACTRIS - In-Situ unit access mechanism,http://envri.eu/ns/supportsAccessTechnology,http://envri.eu/ns/HTTP,NaN
7,SDN SeaDataNet Central Data Products access mechanism,http://envri.eu/ns/supportsAccessTechnology,http://envri.eu/ns/CAS,NaN
8,ACTRIS ACTRIS-ACCESS access mechanism,http://envri.eu/ns/supportsAccessTechnology,http://envri.eu/ns/HTTP,NaN
9,EPOS local EU-EIDA access mechanism,http://envri.eu/ns/supportsAccessTechnology,http://envri.eu/ns/webService,NaN


In [17]:
q = query("""
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX envri: <http://envri.eu/ns/>
PREFIX rm: <http://www.oil-e.net/ontology/envri-rm.owl#>
PREFIX fairterms: <https://w3id.org/fair/principles/terms/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

SELECT ?l ?p ?o ?li WHERE {
    ?p envri:relatesTo fairterms:A1.2 .
    ?s ?p ?o .
    ?s skos:altLabel ?l .
    OPTIONAL { ?o a rdf:Bag . ?o rdf:li ?li }
    FILTER (?o != envri:NULL)
}
""")

display(q)
write(q, 'output-A1_2.xlsx')

,l,p,o,li
0,EPOS Terradue access mechanism,http://envri.eu/ns/contentAccessAuthorizationRequired,authorization for accessing content needed,NaN
1,SDN SeaDataNet Common DAta Index (CDI) access mechanism,http://envri.eu/ns/contentAccessAuthorizationRequired,false,NaN
2,LW (Marine) EUROBIS access mechanism,http://envri.eu/ns/contentAccessAuthorizationRequired,false,NaN
3,EMSO EMSO data access mechanism,http://envri.eu/ns/contentAccessAuthorizationRequired,false,NaN
4,ACTRIS EARLINET Database access mechanism,http://envri.eu/ns/contentAccessAuthorizationRequired,true,NaN
5,LW (Marine) Marine Data Archive access mechanism,http://envri.eu/ns/contentAccessAuthorizationRequired,false,NaN
6,SDN SeaDataNet Central Data Products access mechanism,http://envri.eu/ns/contentAccessAuthorizationRequired,false,NaN
7,Euro-Argo Euro-Argo Data access mechanism,http://envri.eu/ns/contentAccessAuthorizationRequired,false,NaN
8,ACTRIS ACTRIS - In-Situ unit access mechanism,http://envri.eu/ns/contentAccessAuthorizationRequired,partially,NaN
9,SDN SeaDataNet Central Data Products access mechanism,http://envri.eu/ns/contentAccessAuthorizationRequired,false,NaN


In [18]:
q = query("""
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX envri: <http://envri.eu/ns/>
PREFIX rm: <http://www.oil-e.net/ontology/envri-rm.owl#>
PREFIX fairterms: <https://w3id.org/fair/principles/terms/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

SELECT ?l ?p ?o ?li WHERE {
    ?p envri:relatesTo fairterms:I2 .
    ?s ?p ?o .
    ?s skos:altLabel ?l .
    OPTIONAL { ?o a rdf:Bag . ?o rdf:li ?li }
    FILTER (?o != envri:NULL)
}
""")

display(q)
write(q, 'output-I2.xlsx')

,l,p,o,li
0,ICOS Carbon Portal vocabularies,http://envri.eu/ns/hasVocabularyIri,http://www.w3.org/ns/prov,NaN
1,SDN SeaDataNet Common DAta Index (CDI) vocabularies,http://envri.eu/ns/hasVocabularyIri,http://standards.iso.org/ittf/PubliclyAvailableStandards/ISO_19139_Schemas/resources/codelist/ML_gmxCodelists.xml,NaN
2,LW (Marine) Marine Data Archive vocabularies,http://envri.eu/ns/hasVocabularyIri,http://www.marinespecies.org/,NaN
3,SDN SeaDataNet Common DAta Index (CDI) vocabularies,http://envri.eu/ns/hasVocabularyIri,http://vocab.nerc.ac.uk/collection/V22/current/,NaN
4,Euro-Argo Euro-Argo Data vocabularies,http://envri.eu/ns/hasVocabularyIri,http://seadatanet.maris2.nl/v_bodc_vocab_v2/search.asp?lib=P06,NaN
5,SDN SeaDataNet Common DAta Index (CDI) vocabularies,http://envri.eu/ns/hasVocabularyIri,https://edmo.seadatanet.org,NaN
6,SDN SeaDataNet Common DAta Index (CDI) vocabularies,http://envri.eu/ns/hasVocabularyIri,https://edmo.seadatanet.org,NaN
7,SDN SeaDataNet Central Data Products vocabularies,http://envri.eu/ns/hasVocabularyIri,http://standards.iso.org/ittf/PubliclyAvailableStandards/ISO_19139_Schemas/resources/codelist/ML_gmxCodelists.xml,NaN
8,SDN SeaDataNet Central Data Products vocabularies,http://envri.eu/ns/hasVocabularyIri,https://edmo.seadatanet.org,NaN
9,SDN SeaDataNet Central Data Products vocabularies,http://envri.eu/ns/hasVocabularyIri,http://standards.iso.org/ittf/PubliclyAvailableStandards/ISO_19139_Schemas/resources/codelist/ML_gmxCodelists.xml,NaN


In [19]:
q = query("""
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX envri: <http://envri.eu/ns/>
PREFIX rm: <http://www.oil-e.net/ontology/envri-rm.owl#>
PREFIX fairterms: <https://w3id.org/fair/principles/terms/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

SELECT ?l ?p ?o ?li WHERE {
    ?p envri:relatesTo fairterms:I3 .
    ?s ?p ?o .
    ?s skos:altLabel ?l .
    OPTIONAL { ?o a rdf:Bag . ?o rdf:li ?li }
    FILTER (?o != envri:NULL)
}
""")

display(q)
write(q, 'output-I3.xlsx')

,l,p,o,li


In [20]:
q = query("""
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX envri: <http://envri.eu/ns/>
PREFIX rm: <http://www.oil-e.net/ontology/envri-rm.owl#>
PREFIX fairterms: <https://w3id.org/fair/principles/terms/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

SELECT ?l ?p ?o ?li WHERE {
    ?p envri:relatesTo fairterms:R1 .
    ?s ?p ?o .
    ?s skos:altLabel ?l .
    OPTIONAL { ?o a rdf:Bag . ?o rdf:li ?li }
    FILTER (?o != envri:NULL)
}
""")

display(q)
write(q, 'output-R1.xlsx')

,l,p,o,li


In [21]:
q = query("""
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX envri: <http://envri.eu/ns/>
PREFIX rm: <http://www.oil-e.net/ontology/envri-rm.owl#>
PREFIX fairterms: <https://w3id.org/fair/principles/terms/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

SELECT ?l ?p ?o ?li WHERE {
    ?p envri:relatesTo fairterms:R1.1 .
    ?s ?p ?o .
    ?s skos:altLabel ?l .
    OPTIONAL { ?o a rdf:Bag . ?o rdf:li ?li }
    FILTER (?o != envri:NULL)
}
""")

display(q)
write(q, 'output-R1_1.xlsx')

,l,p,o,li
0,EPOS Terradue access mechanism,http://envri.eu/ns/dataLicenseIri,https://creativecommons.org/licenses/,NaN
1,SDN SeaDataNet Common DAta Index (CDI) access mechanism,http://envri.eu/ns/dataLicenseIri,https://www.seadatanet.org/Data-Access/License/1.0,NaN
2,ACTRIS EARLINET Database access mechanism,http://envri.eu/ns/dataLicenseIri,http://actris.nilu.no/Content/Documents/DataPolicy.pdf,NaN
3,SDN SeaDataNet Central Data Products access mechanism,http://envri.eu/ns/dataLicenseIri,https://www.seadatanet.org/Data-Access/License/1.0,NaN
4,Euro-Argo Euro-Argo Data access mechanism,http://envri.eu/ns/dataLicenseIri,https://creativecommons.org/licenses/by/4.0/,NaN
5,ACTRIS ACTRIS - In-Situ unit access mechanism,http://envri.eu/ns/dataLicenseIri,https://ebas-submit.nilu.no/Data-Policy,NaN
6,SDN SeaDataNet Central Data Products access mechanism,http://envri.eu/ns/dataLicenseIri,https://www.seadatanet.org/Data-Access/License/1.0,NaN
7,ACTRIS ACTRIS-ACCESS access mechanism,http://envri.eu/ns/dataLicenseIri,http://actris.nilu.no/Content/Documents/DataPolicy.pdf,NaN
8,ICOS Carbon Portal access mechanism,http://envri.eu/ns/dataLicenseIri,https://creativecommons.org/licenses/by/4.0/,NaN
9,SDN SeaDataNet Common DAta Index (CDI) access mechanism,http://envri.eu/ns/dataLicenseIri,https://www.seadatanet.org/Data-Access/License/1.0,NaN


In [22]:
q = query("""
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX envri: <http://envri.eu/ns/>
PREFIX rm: <http://www.oil-e.net/ontology/envri-rm.owl#>
PREFIX fairterms: <https://w3id.org/fair/principles/terms/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

SELECT ?l ?p ?o ?li WHERE {
    ?p envri:relatesTo fairterms:R1.2 .
    ?s ?p ?o .
    ?s skos:altLabel ?l .
    OPTIONAL { ?o a rdf:Bag . ?o rdf:li ?li }
    FILTER (?o != envri:NULL)
}
""")

display(q)
write(q, 'output-R1_2.xlsx')

,l,p,o,li
0,LW (Marine) Marine Data Archive metadata,http://envri.eu/ns/hasMachineReadableProvenance,false,NaN
1,SDN SeaDataNet Central Data Products metadata,http://envri.eu/ns/hasMachineReadableProvenance,false,NaN
2,SDN SeaDataNet Common DAta Index (CDI) metadata,http://envri.eu/ns/hasMachineReadableProvenance,false,NaN
3,EPOS Terradue metadata,http://envri.eu/ns/hasMachineReadableProvenance,false,NaN
4,SDN SeaDataNet Common DAta Index (CDI) metadata,http://envri.eu/ns/hasMachineReadableProvenance,false,NaN
5,ACTRIS ACTRIS-ACCESS metadata,http://envri.eu/ns/hasMachineReadableProvenance,false,NaN
6,SDN SeaDataNet Central Data Products metadata,http://envri.eu/ns/hasMachineReadableProvenance,false,NaN
7,ACTRIS ACTRIS - In-Situ unit metadata,http://envri.eu/ns/hasMachineReadableProvenance,false,NaN
8,ACTRIS EARLINET Database metadata,http://envri.eu/ns/hasMachineReadableProvenance,true,NaN
9,ICOS Carbon Portal metadata,http://envri.eu/ns/hasMachineReadableProvenance,false,NaN


In [23]:
q = query("""
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX envri: <http://envri.eu/ns/>
PREFIX rm: <http://www.oil-e.net/ontology/envri-rm.owl#>
PREFIX fairterms: <https://w3id.org/fair/principles/terms/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

SELECT ?l ?p ?o ?li WHERE {
    ?p envri:relatesTo fairterms:R1.3 .
    ?s ?p ?o .
    ?s skos:altLabel ?l .
    OPTIONAL { ?o a rdf:Bag . ?o rdf:li ?li }
    FILTER (?o != envri:NULL)
}
""")

display(q)
write(q, 'output-R1_3.xlsx')

,l,p,o,li
0,Euro-Argo Data data management plans,http://envri.eu/ns/hasComplianceValidationService,true,NaN
1,Marine Data Archive data management plans,http://envri.eu/ns/hasComplianceValidationService,false,NaN
2,EMSO data data management plans,http://envri.eu/ns/hasComplianceValidationService,false,NaN
3,EARLINET Database data management plans,http://envri.eu/ns/hasComplianceValidationService,false,NaN
4,SeaDataNet Central Data Products data management plans,http://envri.eu/ns/hasComplianceValidationService,false,NaN
5,EUROBIS data management plans,http://envri.eu/ns/hasComplianceValidationService,false,NaN
6,SeaDataNet Central Data Products data management plans,http://envri.eu/ns/hasComplianceValidationService,false,NaN
7,Carbon Portal data management plans,http://envri.eu/ns/hasComplianceValidationService,false,NaN
8,ACTRIS - In-Situ unit data management plans,http://envri.eu/ns/hasComplianceValidationService,false,NaN
9,SeaDataNet Common DAta Index (CDI) data management plans,http://envri.eu/ns/hasComplianceValidationService,false,NaN


In [24]:
q = query("""
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX envri: <http://envri.eu/ns/>
PREFIX rm: <http://www.oil-e.net/ontology/envri-rm.owl#>
PREFIX fairterms: <https://w3id.org/fair/principles/terms/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

SELECT ?l ?p ?o ?li WHERE {
    ?p envri:relatesTo fairterms:F1 .
    ?s ?p ?o .
    ?s skos:altLabel ?l .
    OPTIONAL { ?o a rdf:Bag . ?o rdf:li ?li }
    FILTER (?o != envri:NULL)
}
""")

display(q)
write(q, 'output-F1.xlsx')

,l,p,o,li
0,SDN SeaDataNet Central Data Products identifier,http://envri.eu/ns/usesIdentifierSystem,http://envri.eu/ns/DigitalObjectIdentifier,NaN
1,EPOS Terradue identifier,http://envri.eu/ns/usesIdentifierSystem,http://envri.eu/ns/None,NaN
2,ICOS Carbon Portal identifier,http://envri.eu/ns/usesIdentifierSystem,http://envri.eu/ns/DigitalObjectIdentifier,NaN
3,SDN SeaDataNet Common DAta Index (CDI) identifier,http://envri.eu/ns/usesIdentifierSystem,http://envri.eu/ns/B2HANDLE,NaN
4,ACTRIS ACTRIS-ACCESS identifier,http://envri.eu/ns/usesIdentifierSystem,http://envri.eu/ns/None,NaN
5,LW (Marine) EUROBIS identifier,http://envri.eu/ns/usesIdentifierSystem,http://envri.eu/ns/DigitalObjectIdentifier,NaN
6,EPOS European Federated Data Archive identifier,http://envri.eu/ns/usesIdentifierSystem,http://envri.eu/ns/ePIC,NaN
7,SDN SeaDataNet Central Data Products identifier,http://envri.eu/ns/usesIdentifierSystem,http://envri.eu/ns/DigitalObjectIdentifier,NaN
8,Euro-Argo Euro-Argo Data identifier,http://envri.eu/ns/usesIdentifierSystem,http://envri.eu/ns/DigitalObjectIdentifier,NaN
9,ICOS Carbon Portal identifier,http://envri.eu/ns/usesIdentifierSystem,http://envri.eu/ns/Handle,NaN


In [25]:
q = query("""
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX envri: <http://envri.eu/ns/>
PREFIX rm: <http://www.oil-e.net/ontology/envri-rm.owl#>
PREFIX fairterms: <https://w3id.org/fair/principles/terms/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

SELECT ?l ?p ?o ?li WHERE {
    ?p envri:relatesTo fairterms:F2 .
    ?s ?p ?o .
    ?s skos:altLabel ?l .
    OPTIONAL { ?o a rdf:Bag . ?o rdf:li ?li }
    FILTER (?o != envri:NULL)
}
""")

display(q)
write(q, 'output-F2.xlsx')

,l,p,o,li
0,EPOS Terradue identifier,http://envri.eu/ns/includesMetadataSchema,http://envri.eu/ns/None,NaN
1,ACTRIS ACTRIS-ACCESS identifier,http://envri.eu/ns/includesMetadataSchema,http://envri.eu/ns/Planned,NaN
2,EPOS European Federated Data Archive identifier,http://envri.eu/ns/includesMetadataSchema,http://envri.eu/ns/None,NaN
3,EPOS local EU-EIDA identifier,http://envri.eu/ns/includesMetadataSchema,http://envri.eu/ns/None,NaN
4,EPOS RESIF (France) identifier,http://envri.eu/ns/includesMetadataSchema,http://envri.eu/ns/None,NaN
5,ICOS Carbon Portal identifier,http://envri.eu/ns/includesMetadataSchema,http://envri.eu/ns/None,NaN
6,EPOS European Federated Data Archive identifier,http://envri.eu/ns/includesMetadataSchema,n531,http://envri.eu/ns/FileLocation
7,EPOS European Federated Data Archive identifier,http://envri.eu/ns/includesMetadataSchema,n531,http://envri.eu/ns/FileChecksum
8,EPOS European Federated Data Archive identifier,http://envri.eu/ns/includesMetadataSchema,n531,http://envri.eu/ns/PIDReplica
9,ACTRIS ACTRIS - In-Situ unit identifier,http://envri.eu/ns/includesMetadataSchema,http://envri.eu/ns/Planned,NaN


In [26]:
q = query("""
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX envri: <http://envri.eu/ns/>
PREFIX rm: <http://www.oil-e.net/ontology/envri-rm.owl#>
PREFIX fairterms: <https://w3id.org/fair/principles/terms/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

SELECT ?l ?p ?o ?li WHERE {
    ?p envri:relatesTo fairterms:F3 .
    ?s ?p ?o .
    ?s skos:altLabel ?l .
    OPTIONAL { ?o a rdf:Bag . ?o rdf:li ?li }
    FILTER (?o != envri:NULL)
}
""")

display(q)
write(q, 'output-F3.xlsx')

,l,p,o,li
0,LW (Marine) Marine Data Archive metadata,http://envri.eu/ns/persistentIdentifiersAreIncluded,true,NaN
1,SDN SeaDataNet Central Data Products metadata,http://envri.eu/ns/persistentIdentifiersAreIncluded,true,NaN
2,SDN SeaDataNet Common DAta Index (CDI) metadata,http://envri.eu/ns/persistentIdentifiersAreIncluded,true,NaN
3,EPOS Terradue metadata,http://envri.eu/ns/persistentIdentifiersAreIncluded,true,NaN
4,Euro-Argo Euro-Argo Data metadata,http://envri.eu/ns/persistentIdentifiersAreIncluded,true,NaN
5,SDN SeaDataNet Common DAta Index (CDI) metadata,http://envri.eu/ns/persistentIdentifiersAreIncluded,true,NaN
6,ACTRIS ACTRIS-ACCESS metadata,http://envri.eu/ns/persistentIdentifiersAreIncluded,false,NaN
7,SDN SeaDataNet Central Data Products metadata,http://envri.eu/ns/persistentIdentifiersAreIncluded,true,NaN
8,EPOS European Federated Data Archive metadata,http://envri.eu/ns/persistentIdentifiersAreIncluded,partially,NaN
9,ACTRIS ACTRIS - In-Situ unit metadata,http://envri.eu/ns/persistentIdentifiersAreIncluded,false,NaN


In [27]:
q = query("""
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX envri: <http://envri.eu/ns/>
PREFIX rm: <http://www.oil-e.net/ontology/envri-rm.owl#>
PREFIX fairterms: <https://w3id.org/fair/principles/terms/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

SELECT ?l ?p ?o ?li WHERE {
    ?p envri:relatesTo fairterms:F4 .
    ?s ?p ?o .
    ?s skos:altLabel ?l .
    OPTIONAL { ?o a rdf:Bag . ?o rdf:li ?li }
    FILTER (?o != envri:NULL)
}
""")

display(q)
write(q, 'output-F4.xlsx')

,l,p,o,li
0,EPOS repository,http://envri.eu/ns/inRegistries,n529,http://envri.eu/ns/LocalRegistry
1,ICOS repository,http://envri.eu/ns/inRegistries,n41,http://envri.eu/ns/re3data
2,LW (Marine) repository,http://envri.eu/ns/inRegistries,http://envri.eu/ns/None,NaN
3,EMSO repository,http://envri.eu/ns/inRegistries,http://envri.eu/ns/None,NaN
4,ACTRIS repository,http://envri.eu/ns/inRegistries,n256,http://envri.eu/ns/WIS
5,ACTRIS repository,http://envri.eu/ns/inRegistries,n256,http://envri.eu/ns/re3data
6,ACTRIS repository,http://envri.eu/ns/inRegistries,n256,http://envri.eu/ns/GEOSS
7,ACTRIS repository,http://envri.eu/ns/inRegistries,http://envri.eu/ns/None,NaN
8,SDN repository,http://envri.eu/ns/inRegistries,n394,http://envri.eu/ns/GEOSS
9,SDN repository,http://envri.eu/ns/inRegistries,n394,http://envri.eu/ns/IODEOceanDataPortal
